# Face search using Amazon Rekognition

***
This notebook provides a walkthrough of [compare faces API](https://docs.aws.amazon.com/rekognition/latest/dg/API_CompareFaces.html) and [face search API](https://docs.aws.amazon.com/rekognition/latest/dg/collections.html) in Amazon Rekognition to detect known faces.
***

In [ ]:
# Initialize dependencies
import boto3
import botocore
from IPython.display import HTML, display, Image as IImage
import time

# Initialize clients
REGION = boto3.session.Session().region_name
rekognition = boto3.client('rekognition', REGION)
s3 = boto3.client('s3', REGION)

%store -r bucket_name

# Detect known faces in image

There are two main ways to detect known faces by using Amazon Rekognition.


1. The first is by using [the compare faces API](https://docs.aws.amazon.com/rekognition/latest/dg/faces-comparefaces.html) to compare two images.
2. The second is by creating a collection of known images using the [index faces API and then using the search faces API](https://docs.aws.amazon.com/rekognition/latest/dg/collections.html) to detect all the faces belonging to a particular collection on an image or video.
***

### 1. Call Rekognition to compare two faces

<https://docs.aws.amazon.com/rekognition/latest/dg/API_CompareFaces.html>

In [ ]:
# Show images
source = "media/identity.jpg" # image profile
target = "media/looking_at_screen.jpg" # screenshot

display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': source})))
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': target})))

In [ ]:
compare_faces_response = rekognition.compare_faces(
    SourceImage={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': source
        }
    },
    TargetImage={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': target
        }        
    })

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see detected faces, confidence score and additional information.

In [ ]:
print("Matched Faces: {}".format(len(compare_faces_response['FaceMatches'])))
display(compare_faces_response)

### 2. Use Index Faces and Search Faces

Now let's detect faces using the Search functionality. 

#### Call Rekognition to create a new collection

<https://docs.aws.amazon.com/rekognition/latest/dg/API_CreateCollection.html>

In [ ]:
collection_id = bucket_name

# Create collection unless it already exsists
try:
    rekognition.describe_collection(CollectionId=collection_id)
    print("Collection already exists")
except botocore.exceptions.ClientError as e:
    rekognition.create_collection(CollectionId=collection_id)
    print("Collection created")

In [ ]:
# Show image
image_name = "media/looking_at_screen.jpg"
display(IImage(url=s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': image_name})))

#### Call Rekognition to search faces in the collection

By making this call, unsurpsingly we'll get no matches the first time, because no image has been indexed yet.

<https://docs.aws.amazon.com/rekognition/latest/dg/API_SearchFacesByImage.html>

In [ ]:
search_faces_response = rekognition.search_faces_by_image(
    CollectionId=collection_id,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see detected faces, confidence score and additional information.

In [ ]:
print("Matched Faces: {}".format(len(search_faces_response['FaceMatches'])))
display(search_faces_response)

#### Index Face in the Collection

Now let's call Amazon Rekognition to create a new identity for the given collection.
<https://docs.aws.amazon.com/rekognition/latest/dg/API_IndexFaces.html>

In [ ]:
index_faces_response = rekognition.index_faces(
    CollectionId=collection_id,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': 'media/identity.jpg',
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see an ID identifying the newly created identity called FaceId.
You can persist that ID in a Database and associate it with a Name for a future lookup.

In [ ]:
identities = {}
identities[index_faces_response['FaceRecords'][0]['Face']['FaceId']] = 'John Doe'
display(index_faces_response)

#### Call Rekognition again to search faces in the collection

This time, the search should match the indexed identity.
<https://docs.aws.amazon.com/rekognition/latest/dg/API_SearchFacesByImage.html>

In [ ]:
search_faces_response = rekognition.search_faces_by_image(
    CollectionId=collection_id,
    Image={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': image_name,
        }
    }
)

#### Review the raw JSON reponse from Rekognition

In the JSON response below, you will see detected faces, confidence score and additional information.

In [ ]:
print("Matched Faces: {}".format(len(search_faces_response['FaceMatches'])))
for match in search_faces_response['FaceMatches']:
    print("Detected {} with Confidence: {}".format(identities[match['Face']['FaceId']], match['Face']['Confidence']))

display(search_faces_response)

# Search faces in video
Search recognition in video is an async operation. 
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartFaceSearch.html. 

- First we start a search detection job which returns a Job Id.
- We can then call `get_search_detection` to get the job status and after job is complete, we can get face metadata.
- In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.
***

In [ ]:
video_name = "media/leaving.mp4"

#### Show video in the player

In [ ]:
s3_video_url = s3.generate_presigned_url('get_object', Params={'Bucket': bucket_name, 'Key': video_name})

video_tag = "<video controls='controls' autoplay width='640' height='360' name='Video' src='{0}'></video>".format(s3_video_url)
video_ui = "<table><tr><td style='vertical-align: top'>{}</td></tr></table>".format(video_tag)
display(HTML(video_ui))


#### Call Rekognition to start a job for face search

In [ ]:
start_search_detection = rekognition.start_face_search(
    CollectionId=collection_id,
    Video={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': video_name,
        }
    },
)

search_job_id = start_search_detection['JobId']
display("Job Id: {0}".format(search_job_id))

### Additional (Optional) Request Attributes

ClientRequestToken:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartFaceSearch.html#rekognition-StartFaceSearch-request-ClientRequestToken

FaceMatchThreshold:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartFaceSearch.html#rekognition-StartFaceSearch-request-FaceMatchThreshold

JobTag:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartFaceSearch.html#rekognition-StartFaceSearch-request-JobTag

NotificationChannel:
https://docs.aws.amazon.com/rekognition/latest/dg/API_StartFaceSearch.html#rekognition-StartFaceSearch-request-NotificationChannel


#### Wait for object detection job to complete

In production use cases, you would usually use StepFunction or SNS topic to get notified when job is complete.

In [ ]:
get_search_detection = rekognition.get_face_search(
    JobId=search_job_id
)

while(get_search_detection['JobStatus'] == 'IN_PROGRESS'):
    time.sleep(5)
    print('.', end='')
 
    get_search_detection = rekognition.get_face_search(
        JobId=search_job_id)
    
display(get_search_detection['JobStatus'])

#### Review raw JSON reponse from Rekognition

In the JSON response below, you will see list of detected faces and attributes.
For each detected face, you will see information like Timestamp.

In [ ]:
display(get_search_detection)

#### Display face detected by timestamp and alert when faces are not detected

In [ ]:
# Faces detected in each frame
prev_ts = 0
threshold = 1000 # ms
for person in get_search_detection['Persons']:
    ts = person["Timestamp"]
    if ts-prev_ts>threshold:
        print("ALERT - no face matched for {} seconds".format((ts-prev_ts)/1000))
    for match in person["FaceMatches"]:
        confidence = match["Face"]["Confidence"]
        identity = identities[match["Face"]["FaceId"]]
        print("Detected {} on Timestamp: {} with confidence: {}".format(identity, ts, confidence))
    prev_ts = ts

#### Cleanup resources

Let's call Rekognition to delete the Collection.
<https://docs.aws.amazon.com/rekognition/latest/dg/API_DeleteCollection.html>

In [ ]:
rekognition.delete_collection(CollectionId=collection_id)

***
### References
- https://docs.aws.amazon.com/rekognition/latest/dg/API_CreateCollection.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_IndexFaces.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_SearchFaces.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_StartFaceSearch.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetLabelDetection.html
- https://docs.aws.amazon.com/rekognition/latest/dg/API_GetFaceSearch.html
***

You have successfully used Amazon Rekognition to search for known faces in images and videos.